## Functions

In this notebook, we lay out functions that we will use in other notebooks to clean, sort, analyze and visualize data.

## Setup

We start by importing libraries and data, as well as tweaking library settings

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from colour import Color
from scipy.interpolate import InterpolatedUnivariateSpline as Spline

pd.options.mode.chained_assignment = None

In [2]:
sipp_2014_2021 = pd.read_csv(r"C:\Users\shell\Python stuff\SIPP data\SIPP_2014_2021.txt", index_col=0)
NPSAS = pd.read_csv(r"C:\Users\shell\Python stuff\NCES DataLab Data\datalab_completed_2015_16.csv")
NPSAS_20 = pd.read_csv(r"C:\Users\shell\Python stuff\NCES DataLab Data\datalab_completed_2019_20.csv")
states = gpd.read_file(r"C:\Users\shell\Python stuff\Geo data\cb_2018_us_state_500k")

C:\Users\shell\AppData\Local\Temp\ipykernel_3424\1531450261.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  sipp_2014_2021 = pd.read_csv(r"C:\Users\shell\Python stuff\SIPP data\SIPP_2014_2021.txt", index_col=0)


## SIPP Data Functions

Functions intended for use with the SIPP data:

In [3]:
def median(dataframe, independent, dependent, point):
    '''
    Finds the median value of the dependent variable at the point given for the independent variable
    '''
    df = dataframe
    df = df[df[independent] == point]
    if df[dependent].median() > 0:
        return df[dependent].median()
    else:
        return -1

In [4]:
def average(list):
    if len(list) != 0:
        return sum(list) / len(list)
    else:
        return 0

def list_avg(dataframe, independent, dependent, point):
    '''
    Finds the average value of the dependent variable at the point given for the independent variable
    '''
    df = dataframe
    df = df[df[independent] == point]
    if average(df[dependent]) > 0:
        return average(df[dependent])
    else:
        return -1

In [5]:
def mean_sort_by(dataframe, independent, dependent):
    '''
    Sorts a dataframe such that it compares the indpendent variable to the mean of the dependent variable
    '''
    df = dataframe
    df = df.fillna(-1)
    df[independent] = df[independent].astype('int')
    df = df[(df['MONTHCODE'] == 1) & (df['EEDUC'] == 43) & (df[dependent] > 0) & (df[independent] > 0)]
    new_df = pd.DataFrame()
    new_df[independent] = range(df[independent].min(), (df[independent].max()+1))
    
    total_list = []
    for x in range(df[independent].min(), (df[independent].max()+1)):
        total_df = df[df[independent] == x]
        total_list.append(sum(total_df[dependent]))
    new_df['total ' + dependent] = total_list
    
    percent_list = []
    for x in range(df[independent].min(), df[independent].max()+1):
        total = sum(new_df['total ' + dependent])
        x_df = new_df[new_df[independent] == x]
        percent_list.append((x_df['total ' + dependent].iloc[0] / total) * 100)
    new_df['percent ' + dependent] = percent_list
    
    median_list = []
    for point in new_df[independent]:
        median_list.append(list_avg(df, independent, dependent, point))

    new_df['mean ' + dependent] = median_list
    
    no_data_independent = []
    for x in range(len(new_df.index)):
        if len(df[(df[independent] == new_df[independent].iloc[x])].index) < 10:
            no_data_independent.append(x)
    new_df.drop(index=no_data_independent, axis='index', inplace=True)

    return new_df

In [6]:
def median_sort_by(dataframe, independent, dependent):
    '''
    Sorts a dataframe such that it compares the indpendent variable to the median of the dependent variable
    '''
    df = dataframe
    df = df.fillna(-1)
    df[independent] = df[independent].astype('int')
    df = df[(df['MONTHCODE'] == 1) & (df['EEDUC'] == 43) & (df[dependent] > 0) & (df[independent] > 0)]
    new_df = pd.DataFrame()
    new_df[independent] = range(df[independent].min(), (df[independent].max()+1))
    
    total_list = []
    for x in range(df[independent].min(), (df[independent].max()+1)):
        total_df = df[df[independent] == x]
        total_list.append(sum(total_df[dependent]))
    new_df['total ' + dependent] = total_list
    
    percent_list = []
    for x in range(df[independent].min(), (df[independent].max()+1)):
        total = sum(new_df['total ' + dependent])
        x_df = new_df[new_df[independent] == x]
        percent_list.append((x_df['total ' + dependent].iloc[0] / total) * 100)
    new_df['percent ' + dependent] = percent_list
    
    median_list = []
    for point in new_df[independent]:
        median_list.append(median(df, independent, dependent, point))

    new_df['median ' + dependent] = median_list
    
    no_data_independent = []
    for x in range(len(new_df.index)):
        if len(df[(df[independent] == new_df[independent].iloc[x])].index) < 10:
            no_data_independent.append(x)
    new_df.drop(index=no_data_independent, axis='index', inplace=True)
    
    new_df[new_df == 0] = np.nan
    new_df[new_df == -1] = np.nan
    
    return new_df

In [7]:
def mean_bar_graph(dataframe, independent, dependent, title, xlab, ylab):
    df = mean_sort_by(dataframe, independent, dependent)
    bars = plt.bar(df[independent], df['mean ' + dependent])
    color_gradient_bars(bars, '#00538f', '#308389')
    plt.title(title)
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    if independent == 'ESEX':
        plt.xticks([1, 2], ['Male', 'Female'])
    plt.show()

In [8]:
def total_bar_graph(dataframe, independent, dependent, title, xlab, ylab):
    df = mean_sort_by(dataframe, independent, dependent)
    bars = plt.bar(df[independent], df['total ' + dependent])
    color_gradient_bars(bars, '#00538f', '#308389')
    plt.title(title)
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    if independent == 'ESEX':
        plt.xticks([1, 2], ['Male', 'Female'])
    plt.show()

In [9]:
def median_spline(dataframe, independent, dependent, title, xlab, ylab):
    
    dataframe = median_sort_by(dataframe, independent, dependent)
    dependent = 'median ' + dependent
    x=np.linspace(dataframe[independent].min(), dataframe[independent].max(), 1000)
    spl = Spline(dataframe[independent], dataframe[dependent])
    plt.plot(x, spl(x))
    
    # fractions have no practical use, they're used for graph cleanliness
    dependent_fraction = (dataframe[dependent].max() - dataframe[dependent].min())*0.05
    independent_fraction = (dataframe[independent].max() - dataframe[independent].min())*0.05
    
    plt.ylim(dataframe[dependent].min() - dependent_fraction, dataframe[dependent].max() + dependent_fraction)
    plt.xlim(dataframe[independent].min() - independent_fraction, dataframe[independent].max() + independent_fraction)
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.title(title)
    
    plt.show()

In [10]:
def median_regression(dataframe, independent, dependent, title, xlab, ylab, degree):
    '''
    Graphs the given dataframe and it's polynomial regression. 
    '''
    dataframe = median_sort_by(dataframe, independent, dependent)
    mdep = 'median ' + dependent
    dataframe = dataframe[dataframe[mdep] > 0]
    
    coefficients = np.polyfit(x=dataframe[independent], y=dataframe[mdep], deg=degree)
    function = np.poly1d(coefficients)
    
    x=np.linspace(dataframe[independent].min(), dataframe[independent].max(), 1000)
    y=function(x)
    plt.plot(x, y, linewidth=4, color='blue')
    
    # fractions have no practical use, they're used for graph cleanliness
    dependent_fraction = (dataframe[mdep].max() - dataframe[mdep].min())*0.05
    independent_fraction = (dataframe[independent].max() - dataframe[independent].min())*0.05
    
    plt.ylim(dataframe[mdep].min() - dependent_fraction, dataframe[mdep].max() + dependent_fraction)
    plt.xlim(dataframe[independent].min() - independent_fraction, dataframe[independent].max() + independent_fraction)
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.title(title)
    
    colors = color_gradient(len(dataframe[independent]), '#00538f', '#308389')
    plt.scatter(x=dataframe[independent], y=dataframe[mdep],
               # s=(dataframe['total ' + dependent] / 1000)
               s=100, color=colors)
    
    plt.show()

In [11]:
def mean_regression(dataframe, independent, dependent, title, xlab, ylab, degree):
    '''
    Graphs the given dataframe and it's polynomial regression. 
    '''
    dataframe = mean_sort_by(dataframe, independent, dependent)
    mdep = 'mean ' + dependent
    dataframe = dataframe[dataframe[mdep] > 0]
    
    coefficients = np.polyfit(x=dataframe[independent], y=dataframe[mdep], deg=degree)
    function = np.poly1d(coefficients)
    
    x=np.linspace(dataframe[independent].min(), dataframe[independent].max(), 1000)
    y=function(x)
    plt.plot(x, y, linewidth=4, color='blue')
    
    # fractions have no practical use, they're used for graph cleanliness
    dependent_fraction = (dataframe[mdep].max() - dataframe[mdep].min())*0.05
    independent_fraction = (dataframe[independent].max() - dataframe[independent].min())*0.05
    
    plt.ylim(dataframe[mdep].min() - dependent_fraction, dataframe[mdep].max() + dependent_fraction)
    plt.xlim(dataframe[independent].min() - independent_fraction, dataframe[independent].max() + independent_fraction)
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.title(title)
    
    colors = color_gradient(len(dataframe[independent]), '#00538f', '#308389')
    plt.scatter(x=dataframe[independent], y=dataframe[mdep],
               # s=(dataframe['total ' + dependent] / 1000)
               s=100, color=colors)
    
    plt.show()

In [12]:
def sipp_clean(df):
    df = df[(df['EEDUC'] == 43) 
            & (df['MONTHCODE'] == 1)
            & (df['TOEDDEBTVAL'] > 0)]
    return df

In [13]:
def group_age(dataframe):
    df = dataframe

    for x in range(len(df.index)):
        if df['TAGE'].iloc[x] < 24:
            df.iloc[x, df.columns.get_loc('TAGE')] = '<25'
        
        elif df['TAGE'].iloc[x] < 35:
            df.iloc[x, df.columns.get_loc('TAGE')] = '25 - 34'
        
        elif df['TAGE'].iloc[x] < 50:
            df.iloc[x, df.columns.get_loc('TAGE')] = '35 - 49'
        
        elif df['TAGE'].iloc[x] < 62:
            df.iloc[x, df.columns.get_loc('TAGE')] = '50 - 61'
        
        else:
            df.iloc[x, df.columns.get_loc('TAGE')] = '>61'
    return df

In [14]:
def age_percent_total(dataframe, independent):
    df = dataframe[dataframe[independent] > 0]
    
    total = sum(df[independent])
    new_df = pd.DataFrame()
    new_df['TAGE'] = ['<25', '25 - 34', '35 - 49', '50 - 61', '>61']
    
    #calculate total and percent of people included in the sample
    total_list = []
    percent_list = []
    total_people = len(df.index)
    for x in ['<25', '25 - 34', '35 - 49', '50 - 61', '>61']:
        total_list.append(len(df[(df['TAGE'] == x)].index))
        percent_list.append((len(df[(df['TAGE'] == x)].index) / total_people) * 100)
    new_df['total people'] = total_list
    new_df['percent of total people'] = percent_list
    
    #calculate total, percent, and average of the independent variable
    total_list = []
    percent_list = []
    mean_list = []
    for x in ['<25', '25 - 34', '35 - 49', '50 - 61', '>61']:
        total_df = df[df['TAGE'] == x]
        percent_list.append((sum(total_df[independent]) / total) * 100)
        total_list.append(sum(total_df[independent]))
        mean_list.append(sum(total_df[independent]) / len(total_df[independent].index))
    new_df['mean ' + independent] = mean_list
    new_df['total ' + independent + ' in database'] = total_list
    new_df['percent of ' + independent] = percent_list
    return new_df

In [15]:
def add_percent_by_age(dataframe, independent, dependent):
    sipp_unfiltered = pd.read_csv(r"C:\Users\shell\Python stuff\SIPP data\SIPP_2014_2021.txt", index_col=0)
    sipp_unfiltered = group_age(sipp_unfiltered[(sipp_unfiltered['EEDUC'] == 43) & (sipp_unfiltered['MONTHCODE'] == 1) & (sipp_unfiltered['YEAR'] == 2021)])
    percent_list = []
    for x in dataframe[independent]:
        df = sipp_unfiltered[sipp_unfiltered[independent] == x]
        percent_list.append(len(df[df[dependent] > 0].index) / 
                           len(df.index) * 100)
    dataframe['Percent of total ' + dependent + ' per ' + independent] = percent_list
    return dataframe

In [16]:
def add_percent(dataframe, independent, dependent):
    sipp_unfiltered = pd.read_csv(r"C:\Users\shell\Python stuff\SIPP data\SIPP_2014_2021.txt", index_col=0)
    sipp_unfiltered = sipp_unfiltered[(sipp_unfiltered['EEDUC'] == 43) & (sipp_unfiltered['MONTHCODE'] == 1) & (sipp_unfiltered['YEAR'] == 2021)]
    percent_list = []
    for x in dataframe[independent]:
        df = sipp_unfiltered[sipp_unfiltered[independent] == x]
        percent_list.append(len(df[df[dependent] > 0].index) / 
                           len(df.index) * 100)
    dataframe['Percent of total ' + dependent + ' per ' + independent] = percent_list
    return dataframe

## NPSAS Data Functions

Functions intended for use with the NCES's NPSAS 2015-2016 and 2019-2020 data:

In [17]:
def datalab_round(dataframe):
    '''
    Rounds all values of the data to remove floating points
    '''
    df = dataframe
    df = df.fillna(-1)
    for x in range(len(df.index)):
        for y in range(2, len(df.columns)):
            df.iloc[x, y] = int(round(float(df.iloc[x, y]), 2))
    df = df.replace(-1, np.nan)
    return df

In [18]:
def datalab_bar_graph(dataframe, category, zeroes, title, xlab, ylab):
    '''
    Graphs a category from the NPSAS data.
    Zeroes is a true or false value indicating
    whether or not to include zeroes in averages.
    '''
    df = dataframe
    df = df.replace(-1, np.nan)
    df = df.replace(0, np.nan)
    
    df = df[df['Category'] == category]
    if zeroes:
        bars = plt.bar(df['Value'], df['Amount still owed on all undergraduate loans ( Average )'])
    elif not zeroes:
        bars = plt.bar(df['Value'], df['Amount still owed on all undergraduate loans ( Average Without Zeros )'])
    
    color_gradient_bars(bars, '#00538f', '#308389')
    plt.title(title)
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.show
    return bars

In [19]:
def states_merge(df):
    # drop states/territories that aren't part of the 48 contiguous states
    df = df.drop([13, 27, 37, 38, 42, 44, 45])
    # sort state names alphabetically
    df = df.sort_values('NAME')
    df = df.reset_index()

    # drop Hawaii and Alaska
    NPSAS_states = NPSAS.drop([115, 125])
    NPSAS_states = datalab_round(NPSAS_states)
    
    state_debt = NPSAS_states[NPSAS_states['Category'] == 'State of legal residence']
    state_debt = state_debt.reset_index()
    df['AVGDEBT'] = state_debt['Amount still owed on all undergraduate loans ( Average Without Zeros )']
    df['AVGDEBTWZ'] = state_debt['Amount still owed on all undergraduate loans ( Average )']
    
    return df

In [20]:
def regions_merge(df):
    # drop states/territories that aren't part of the 48 contiguous states
    df = df.drop([13, 27, 37, 38, 42, 44, 45])
    # sort state names alphabetically
    df = df.sort_values('NAME')
    df = df.reset_index()
    
    # drop Hawaii and Alaska from the student debt data
    NPSAS_states = NPSAS_20.drop([115, 125])
    NPSAS_states = datalab_round(NPSAS_states)
    
    # create a new dataframe with states and regions
    regions = pd.DataFrame()
    regions['region'] = ['New England', 'Mideast', 'Great Lakes', 'Plains', 
                         'South East', 'Southwest', 'Rocky Mountains', 'Far West']
    regions['states'] = [['CT', 'ME', 'MA', 'NH', 'RI', 'VT'], 
                         ['DE', 'DC', 'MD', 'NJ', 'NY', 'PA'], 
                         ['IL', 'IN', 'MI', 'OH', 'WI'], 
                         ['IA', 'KS', 'MN', 'MO', 'NE', 'ND', 'SD'], 
                         ['AL', 'AR', 'FL', 'GA', 'KY', 'LA', 'MS', 'NC', 'SC', 'TN', 'VA', 'WV'], 
                         ['AZ', 'NM', 'OK', 'TX'],
                         ['CO', 'ID', 'MT', 'UT', 'WY'], 
                         ['CA', 'NV', 'OR', 'WA']]
    
    # add debt per region to the regions dataframe
    regions_debt = NPSAS_states[NPSAS_states['Category'] == 'NPSAS institution region'].reset_index()
    regions_debt = roundup(regions_debt['Amount still owed on all undergraduate loans ( Average Without Zeros )'], 100)
    regions['average debt'] = regions_debt
    
    # for each state in the regions dataframe: finds it in the geodataframe and adds its region and average debt
    df['REGION'] = ''
    df['AVGDEBTREG'] = ''
    i = 0
    for states in regions['states']:
        for state in states:
            state_df = df.loc[df['STUSPS'] == state]
            state_df['REGION'] = regions.loc[i,'region']
            state_df['AVGDEBTREG'] = regions.loc[i, 'average debt']
            df.loc[df['STUSPS'] == state] = state_df
        i = i + 1
    
    # return the dataframe with new data added
    return df

In [21]:
def datalab_graph_geo(lod, cmap):
    if lod == 'state':
        df = states_merge(states)
        ax = df.plot(column='AVGDEBT',
                legend=True, 
                legend_kwds={"orientation": "vertical"},
                missing_kwds={'color': 'lightgrey'}, 
                cmap=cmap)
        ax.set_axis_off()
        # ax.set_title('Average Student Debt by Student\'s State of Residence', fontdict={'fontsize': 12})
        
    elif lod == 'region':
        df = regions_merge(states)
        ax = df.plot(column='AVGDEBTREG',
                legend=True,
                legend_kwds={"loc": "lower right"},
                cmap=cmap)
        ax.set_axis_off()
        # ax.set_title('Average Student Debt by Region of Institution Attended', fontdict={'fontsize': 12})
        
    else:
        return 'Please input an lod of either state or region'
    
    plt.grid(False)
    plt.axis('off')
    plt.show()

## Graphing Functions

Functions intended for graphing use:

In [22]:
def color_one(x, num, color):
    for x in range(x):
        if x == num:
            bars[x].set_color(color)
        else:
            bars[x].set_color('gray')

In [23]:
def color_gradient_bars(bars, c1, c2):
    c1 = Color(c1)
    c2 = Color(c2)
    color_gradient = list(c1.range_to(c2, (len(bars)+2)))
    for x in range(len(bars)):
        color = str(color_gradient[x+1]).lstrip('#')
        rgb = tuple(int(color[i:i+2], 16) for i in (0, 2, 4))
        rgb = [(x / 255) for x in rgb]
        bars[x].set_color(rgb)

In [24]:
def color_gradient(length, c1, c2):
    c1 = Color(c1)
    c2 = Color(c2)
    color_gradient = list(c1.range_to(c2, (length+2)))
    rgb_list = []
    for x in range(length):
        color = str(color_gradient[x+1]).lstrip('#')
        rgb = tuple(int(color[i:i+2], 16) for i in (0, 2, 4))
        rgb = [(x / 255) for x in rgb]
        rgb_list.append(rgb)
        
    
    return rgb_list

In [25]:
def roundup(x, val):
    return round(x/val, 0)*val